In [ ]:
### This project was tested using both hierarchical clustering and Random Forest Regression for 
### selecting sensors used in prediction (dimensionality reduction), but tests conducted with
### Random Forest Regression achieve significantly better results


from numpy.random import seed
seed(178)

import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import pickle
import sys

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model

from matplotlib import pyplot as plt

import numpy as np
from math import sqrt

import os

In [ ]:
# Finds the cluster items for a given column (sensor)

def return_cluster_from_col(clusters_list: list) -> list:
    cluster_index = None
    for i, cluster in enumerate(clusters_list):
        if col in cluster:
            cluster_index = i
      
  
    cluster_from_col = [col]
    for item in clusters_list[cluster_index]:
        if item != col:
            cluster_from_col.append(item)
    return cluster_from_col

In [ ]:
# Finds the "other" sensors needed for predicting, using either clustering or feature importance

def get_supporting_columns(index_name:str, clustering_algorithm: bool) -> list:
    if clustering_algorithm:
        with open(file_path + 'clusters_list.pkl', 'rb') as f:
            clusters_list = pickle.load(f)
        return [index_name] + return_cluster_from_col(clusters_list)
  
    else:
        with open(file_path + 'importance_new/' + col + '_rank.pkl', 'rb') as f:
            rank = pickle.load(f)[:11]
        column_rank = []
        for r in rank:
            column_rank.append(df_temp.columns.values[r])
        return [index_name] + column_rank

In [ ]:
def load_and_scale_data(cols_to_use: list):
    df_load = pd.read_csv(file_path + 'interpolated_no_na_no_noise.csv', usecols=cols_to_use, sep=";")
    df_load = df_load.loc[:, cols_to_use]
    df_load.set_index('timestamp', inplace=True)
    values = df_load.values
    df_load = 0
    scaled_values = scaler.fit_transform(values)
    return scaled_values

In [ ]:
# Creates 3 timesteps for the columns used for prediction

def prepare_data(scaled_values: list) -> pd.DataFrame:
    df_prep = pd.DataFrame(scaled_values)
    columns = np.roll(df_prep.columns.values, -1)
    df_prep = df_prep[columns]
  
    cols, names = list(), list()
  
    for i in range(lag_time_steps-1, -1, -1):
        if (i == 0):
            names += [('var%d(t)' % (j+1)) for j in df_prep.columns.values]
            cols.append(df_prep.shift(i))
        else:
            names += [('var%d(t-%d)' % (j+1, i)) for j in columns[:-1]]
            cols.append(df_prep[columns[:-1]].shift(i))
    
    prepared_data = pd.concat(cols, axis=1)
    prepared_data.columns = names
    prepared_data.dropna(inplace=True)
    return prepared_data

In [ ]:
# Reshape because LSTM needs 3D input

def get_train_and_test_data_reshaped(prepared_data: pd.DataFrame, n_features: int):
    train = prepared_data.sample(frac=0.9, random_state=178)
    test = prepared_data.drop(train.index)
  
    train, test = train.values, test.values
  
  
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
  
  
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], lag_time_steps, n_features))
    test_X = test_X.reshape((test_X.shape[0], lag_time_steps, n_features))
  
    return train_X, train_y, test_X, test_y

In [ ]:
def get_model(train_X):
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam', metrics=['mse', 'mape'])
  
    print(model.summary())
    return model

In [ ]:
def train_model(model_path, model, train_X, train_y, relative_path):
    early_stopping_monitor = EarlyStopping(monitor='loss', mode='min', patience=1, verbose=1)

    checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    history = model.fit(train_X, train_y, epochs=n_epochs, batch_size=n_batch,
                        validation_split=0.1, verbose=2,
                        callbacks=[checkpoint])
                        #callbacks=[early_stopping_monitor, checkpoint])
                        #callbacks=[early_stopping_monitor])
    with open(relative_path + 'history_' + str(n_epochs) + 'epochs.pkl', 'wb') as f_history:
        pickle.dump(history, f_history, pickle.HIGHEST_PROTOCOL)
  
    return history

In [ ]:
def predict_and_inverse_normalization(model, test_X, test_y, n_features, n_observations):
  
    # Make prediction
    prediction = model.predict(test_X, verbose=1)
  
    print('feat', n_features)
    print('obs', n_observations)
    print('lag', lag_time_steps)
  
    # Invert prediction
    test_X = test_X.reshape((test_X.shape[0], n_observations))
  
    concated_prediction = np.concatenate((prediction, test_X[:, -n_features:]), axis=1)
    inverse_prediction = scaler.inverse_transform(concated_prediction)[:, 0]
  
    # Invert scaling for actual values
    test_y = test_y.reshape((len(test_y), 1))
    concated_actual = np.concatenate((test_y, test_X[:, -n_features:]), axis=1)
    inverse_actual = scaler.inverse_transform(concated_actual)[:, 0]
  
    return inverse_prediction, inverse_actual

In [ ]:
def plot(prediction, actual, history, save: bool, relative_path: str, rmse: float, is_clustering: bool):
  
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.set_xlabel('Epoch')
    if is_clustering:
        ax.set_title('Loss - Clustering')
    else:
        ax.set_title('Loss - Feature Importance')
    ax.plot(history.history['loss'], label='Training')
    ax.plot(history.history['val_loss'], label='Validation')
    plt.legend()
    plt.show()
  
    fig2 = plt.figure(facecolor='white')
    ax2 = fig2.add_subplot(111)
    ax2.set_xlabel('Epoch')
    if is_clustering:
        ax2.set_title('MAPE - Clustering')
    else:
        ax2.set_title('MAPE - Feature Importance')
    ax2.plot(history.history['mean_absolute_percentage_error'], label='Mean Absolute Percentage Error')
    plt.legend()
    plt.show()
  
    fig3 = plt.figure(facecolor='white')
    ax3 = fig3.add_subplot(111)
    ax3.set_xlabel('Epoch')
    if is_clustering:
        ax3.set_title('MSE - Clustering')
    else:
        ax3.set_title('MSE - Feature Importance')
  
    ax3.plot(history.history['mean_squared_error'], label='Mean Squared Error')
    plt.legend()
    plt.show()
  
    fig4 = plt.figure(facecolor='white')
    ax4 = fig4.add_subplot(111)
    ax4.set_xlabel('Timestamp Index')
    if is_clustering:
        ax4.set_title('Clustering - RMSE = ' + str(rmse))
    else:
        ax4.set_title('Feature Importance - RMSE = ' + str(rmse))
    ax4.plot(actual, label='Actual Data')
    ax4.plot(prediction, label='Prediction')
    plt.legend()
    plt.show()
  
    fig5 = plt.figure(facecolor='white')
    ax5 = fig5.add_subplot(111)
    ax5.plot(actual, label='True Data')
    plt.legend()
    plt.show()
  
    if save:
      
        loss_path = relative_path + 'loss_' + str(n_epochs) + '_epochs.png'
        fig.savefig(loss_path)

        mape_path = relative_path + 'mape_' + str(n_epochs) + '_epochs.png'
        fig2.savefig(mape_path)

        mse_path = relative_path + 'mse_' + str(n_epochs) + '_epochs.png'
        fig3.savefig(mse_path)

        pred_path = relative_path + 'prediction_' + str(n_epochs) + '_epochs.png'
        fig4.savefig(pred_path)
    
        if not is_clustering:
            actual_path = relative_path + 'actual_' + str(n_epochs) + '_epochs.png'
            fig5.savefig(actual_path)

In [ ]:
def run_full(cols_to_use: list, relative_path: str):
    n_features = len(cols_to_use) - 2
    n_observations = n_features * lag_time_steps
  
    model_path = relative_path + 'model_' + str(n_epochs) + '_epochs.h5'

    scaled_values = load_and_scale_data(cols_to_use)
    prepared_data = prepare_data(scaled_values)
    train_X, train_y, test_X, test_y = get_train_and_test_data_reshaped(
        prepared_data, n_features)

    model = get_model(train_X)

    history = train_model(model_path, model, train_X, train_y, relative_path)
  
    model = load_model(model_path)
  
    history_path = relative_path + 'history_' + str(n_epochs) + 'epochs.pkl'
    with open(history_path, 'rb') as f_history:
        history = pickle.load(f_history)

    prediction, actual = predict_and_inverse_normalization(
        model, test_X, test_y, n_features, n_observations)


    return prediction, actual, history

In [ ]:
def run_full_cluster_and_importance(index_name: str, relative_path: str, save_plot: bool, start_from_cluster: bool):
    predictions = []
    actual_values = []
    histories = []
  
    for i in range(1):
        is_clustering = False
        if start_from_cluster:
            start_from_cluster = False
            continue
        
        if i == 0:
            cols_to_use = get_supporting_columns(index_name, False)
      
        else:
            is_clustering = True
            cols_to_use = get_supporting_columns(index_name, True)
            relative_path += 'clustering_'
    
        print('cols to use')
        print(cols_to_use)
      
        prediction, actual, history = run_full(cols_to_use, relative_path)
        predictions.append(prediction)
        actual_values.append(actual)
        histories.append(history)
    
        rmse = sqrt(mean_squared_error(prediction, actual))

        print('Test RMSE: %.3f' % rmse)
        print(prediction)
        print(actual)
    
        plot(prediction, actual, history, save_plot, relative_path, rmse, is_clustering)
    
    return predictions, actual_values, histories

In [ ]:
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

df = pd.read_csv('interpolated_no_na_no_noise.csv', index_col=0, sep=";")


lag_time_steps = 3

n_epochs = 200
n_batch = 72 

start_from_cluster = False # Incase the feature importance worked well, but clustering needs to be run again


dir_cols = df.columns.values

predictions_both_methods = [] # Both methods referring to clustering and feature importance
actual_values_both_methods = []
histories_both_methods = []


for col in dir_cols:
  
    print('The col is ' + str(col))

    relative_path = file_path + 'models/plots/' + str(col) + '/'

    save_plot = True

    prediction_both_methods, actual_value_both_methods, history_both_methods = 
    run_full_cluster_and_importance(df_temp.index.name, relative_path, save_plot, start_from_cluster)
    
    predictions_both_methods.append(prediction_both_methods)
    actual_values_both_methods.append(actual_value_both_methods)
    histories_both_methods.append(history_both_methods)
  
    print('Finished col ' + str(col))
    print('-----------------------------------------------------------------')
    print('-----------------------------------------------------------------')
    print('-----------------------------------------------------------------')